# DB 초기화 프로그램

In [111]:
!pip install pymysql > /dev/null

In [112]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql (3).json


In [113]:
import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)    # dictionary unpacking

In [114]:
import random
random.seed(2021)

## products 테이블

In [115]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [116]:
cur = conn.cursor()
cur.execute(sql)

0

In [117]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품',
                 '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV',
             '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리',
                '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [118]:
for item in camping_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '캠핑', cost)

텐트 95600 캠핑 87000
타프 97800 캠핑 92910
천막 65700 캠핑 58470
침낭 35300 캠핑 33540
매트 13500 캠핑 12420
랜턴 58500 캠핑 54990
취사용품 16500 캠핑 14850
아이스박스 37600 캠핑 33460
보조배터리 58600 캠핑 50400
워터저그 26500 캠핑 23060


In [119]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

## sales 테이블

In [120]:
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [121]:
cur = conn.cursor()
cur.execute(sql)

0

In [122]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['멀티캠퍼스','한경아카데미','표준협회','생산성본부','테크브루']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-27 표준협회 12 9
2020-1-3 멀티캠퍼스 16 19
2020-1-15 한경아카데미 29 5
2020-1-21 생산성본부 23 9
2020-1-14 한경아카데미 31 14
2020-1-5 테크브루 37 16
2020-1-30 표준협회 14 12
2020-1-1 생산성본부 40 18
2020-1-10 표준협회 31 4
2020-1-3 테크브루 13 14


In [123]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11,40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

## 두 테이블을  Join하기 위한 View 생성

In [124]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [125]:
cur = conn.cursor()
cur.execute(sql)

0

- 데이터를 sales_book.csv로 저장

In [126]:
sql = 'select * from sales_book;'
cur.execute(sql)
results = cur.fetchall()

In [128]:
import pandas as pd

df = pd.DataFrame(results)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       120 non-null    int64 
 1   1       120 non-null    object
 2   2       120 non-null    object
 3   3       120 non-null    int64 
 4   4       120 non-null    object
 5   5       120 non-null    object
 6   6       120 non-null    int64 
 7   7       120 non-null    int64 
 8   8       120 non-null    int64 
 9   9       120 non-null    int64 
 10  10      120 non-null    int64 
dtypes: int64(7), object(4)
memory usage: 10.4+ KB


,0,1,2,3,4,5,6,7,8,9,10
0,104,2020-01-01,표준협회,34,벨트,헬스용품,87400,11,76910,961400,115390
1,109,2020-01-02,테크브루,23,하이패스,자동차용품,48100,17,46660,817700,24480
2,108,2020-01-02,멀티캠퍼스,24,후방카메라,자동차용품,50400,11,48380,554400,22220
3,102,2020-01-06,생산성본부,16,랜턴,캠핑용품,94600,12,89870,1135200,56760
4,103,2020-01-07,한경아카데미,13,천막,캠핑용품,78900,4,75740,315600,12640


In [129]:
del df[3]
del df[0]

In [130]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-01,표준협회,벨트,헬스용품,87400,11,76910,961400,115390
1,2020-01-02,테크브루,하이패스,자동차용품,48100,17,46660,817700,24480
2,2020-01-02,멀티캠퍼스,후방카메라,자동차용품,50400,11,48380,554400,22220
3,2020-01-06,생산성본부,랜턴,캠핑용품,94600,12,89870,1135200,56760
4,2020-01-07,한경아카데미,천막,캠핑용품,78900,4,75740,315600,12640


In [131]:
df.to_csv('sales_book.csv', index=False)

## 마무리

In [132]:
cur.close()
conn.close()